# Sklearn
Download the model

In [1]:
!pip install sklearn
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y = datasets.load_iris(return_X_y=True)
clf.fit(X, y)
from joblib import dump, load
dump(clf, 'iris.joblib')


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 81.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 63.0 MB/s eta 0:00:0000:0100:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=9288b7b77163ac294ae4fbb0031468fdcc96ab0f3a2bc51234f33c1cc1fbb544
  Stored in directory: /home/cc/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


['iris.joblib']

In [2]:
# import job
!mc mb minio/minio-seldon/iris -p
!mc cp ./iris.joblib minio/minio-seldon/iris/

Bucket created successfully `minio/minio-seldon/iris`.
...is.joblib:  5.33 KiB / 5.33 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 732.17 KiB/s 0s

# Rclone parameters
We pass the rclone parameters for getting data from minio using secret

In [3]:
%%writefile rclone-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-rclone-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Writing rclone-secret.yaml


In [4]:
!kubectl apply -f ./rclone-secret.yaml

secret/seldon-rclone-secret created


In [ ]:
%%writefile sklearn.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn-default-init-container
spec:
  name: iris
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/v1.14.0-dev/sklearn/iris
      envSecretRefName: seldon-rclone-secret
      name: classifier
    name: default
    replicas: 1